<a href="https://colab.research.google.com/github/morescode-pm/kaggle-speciesnet/blob/test-speciesnet-raw/run_speciesnet_on_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SpeciesNet Colab demo

This notebook:

* Installs the speciesnet package
* Downloads the model
* Copies it to a writeable folder before using it
* Downloads two test images from this repo
* Processes them using both the CLI (i.e., python -m speciesnet.scripts.run_model) and Python code

As is often the case when you !pip install a library on Colab, you will likely have to run the first code cell (which just does "!pip install speciesnet"), then re-start your Colab runtime, then you should be able to run the whole notebook.


## Imports


In [ ]:
!pip install speciesnet

In [ ]:
from IPython.display import display
from IPython.display import JSON

# from speciesnet import DEFAULT_MODEL
from speciesnet import draw_bboxes
from speciesnet import load_rgb_image
from speciesnet import SpeciesNet
from speciesnet import SUPPORTED_MODELS

In [ ]:
def print_predictions(predictions_dict: dict) -> None:
    print("Predictions:")
    for prediction in predictions_dict["predictions"]:
        print(prediction["filepath"], "=>", prediction["prediction"])

## Download model

In [ ]:
import kagglehub
import os
import shutil

# Choose the folder we're going to download to
model_path = '/content/models'
os.makedirs(model_path, exist_ok=True)

# Download the model (it will go to a folder like /kaggle/input/...)
download_path = kagglehub.model_download('google/speciesnet/PyTorch/v4.0.1a',
                                          force_download=True)

print('Model downloaded to temporary folder: {}'.format(download_path))

# List the contents of the downloaded directory to identify the actual files/subdirectories
model_files = os.listdir(download_path)

# Copy the contents of the model file to our destination folder
for item_name in model_files:
    source_path = os.path.join(download_path, item_name)
    destination_path = os.path.join(model_path, item_name)
    if os.path.isfile(source_path):
        shutil.copy2(source_path, destination_path)
    elif os.path.isdir(source_path):
        shutil.copytree(source_path, destination_path, dirs_exist_ok=True)

print('{} files copied to: {}'.format(len(model_files),model_path))

## Download sample images

In [ ]:
os.makedirs('/content/images',exist_ok=True)
!wget "https://github.com/google/cameratrapai/blob/main/test_data/african_elephants.jpg?raw=true" -O "/content/images/african_elephants.jpg"
!wget "https://github.com/google/cameratrapai/blob/main/test_data/american_black_bear.jpg?raw=true" -O "/content/images/american_black_bear.jpg"

## Run the model using the CLI

In [ ]:
!python -m speciesnet.scripts.run_model --folders '/content/images' --model $model_path --predictions_json '/content/predictions-ensemble.json'

In [ ]:
# print the contents of the output json
import json
with open('/content/predictions-ensemble.json','r') as f:
  d = json.load(f)
print(str(d))


## Run the model using Python code

In [ ]:
model = SpeciesNet(model_path)

In [ ]:
predictions_dict = model.predict(
    instances_dict={
        "instances": [
            {
                "filepath": "/content/images/african_elephants.jpg",
                "latitude": -1.0,
                "longitude": 35.0,
            },
            {
                "filepath": "/content/images/american_black_bear.jpg",
                "country": "USA",
            },
        ]
    }
)

print_predictions(predictions_dict)
display(JSON(predictions_dict))

## Display detections

In [ ]:
first_prediction_item = predictions_dict["predictions"][0]
img = load_rgb_image(first_prediction_item["filepath"])
detections = first_prediction_item["detections"]
img.thumbnail(size=(800, 800))
draw_bboxes(img, detections)